In [1]:
import requests
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import pandas as pd
from xml.etree import ElementTree
import requests

In [ ]:
# # in the perfect world part
# dom = ElementTree.parse("pubmed22n0001.xml")
# Articles = dom.findall('PubmedArticle/MedlineCitation/Article')
# pubmedID = dom.findall('PubmedArticle/PubmedData/ArticleIdList/')
# allTitles = []
# allAuthors = []
# # indexes = []
# IDs = []

# for i, eachArticles in enumerate(Articles):
#     # index.append(i+1)
#     title = eachArticles.find('ArticleTitle').text
#     allTitles.append(title)
#     allArticleAuthors = []
#     # for eachId in eachArticles.findall()
#     for authors in eachArticles.findall('AuthorList/Author'):
#         if authors.find('LastName') is not None:
#             LastName = authors.find('LastName').text
#         else:
#             LastName = "NULL"
#         if authors.find('ForeName') is not None:
#             ForeName = authors.find('ForeName').text
#         else:
#             ForeName = "NULL"
#         allArticleAuthors.append(ForeName + " " + LastName)
#     allAuthors.append(allArticleAuthors)
# for i, each in enumerate(pubmedID):
#     # indexes.append(i+1)
#     if (each.attrib == {'IdType': 'pubmed'}) is not None:
#         if each.attrib == {'IdType': 'pubmed'}:
#             IDs.append(each.text)
#             # indexes.append(i+1)
#     else:
#         IDs.append("NULLID")
#         # indexes.append(i+1)

# components = {
#     'Title' : allTitles,
#     'Author': allAuthors,
#     # 'Indexes' : indexes,
#     'PubMedIDs' : IDs
# }

# OriDf = pd.DataFrame(components)
# customString = "https://pubmed.ncbi.nlm.nih.gov/"
# endCString = "/citations/"
# styles = ["ama", "apa", "mla", "nlm"]

# ama, apa, mla, nlm = [], [], [], []
# # for i, eachId in enumerate(IDs):
# errorList = []
# for eachId in IDs:
#     try:
#         tempMatch = requests.get(customString + eachId + endCString).text
#         matchDict = eval(tempMatch)
#         try:
#             ama.append(matchDict["ama"]['orig'])
#         except:
#             ama.append("NULL")
#         try:
#             apa.append(matchDict["apa"]['orig'])
#         except:
#             apa.append("NULL")
#         try:
#             mla.append(matchDict["mla"]['orig'])
#         except:
#             mla.append("NULL")
#         try:
#             nlm.append(matchDict["nlm"]['orig'])
#         except:
#             nlm.append("NULL")
#     except:
#         errorList.append(eachId)

In [2]:
# Real life part
customString = "https://pubmed.ncbi.nlm.nih.gov"
refString = "references/"
holderIDs = [x for x in range(30000)]
citedByHrefDict = defaultdict(list)
citedString = defaultdict(list)
realisticCitation = defaultdict(list)

for each in holderIDs:
    try:
        tempRawPage = requests.get(customString + "/" + str(each) + "/").text
        soup = bs(tempRawPage, "html.parser")
        tempCitedByList = soup.find('div', class_= 'citedby-articles').find_all('a', class_= 'docsum-title')
        for i, eachCitation in enumerate(tempCitedByList):
            citedByHrefDict[each].append(eachCitation['href'])
            # take max 3 references per article or adjust to your desired amount
            # the more x references, the more columns to be added to the df,
            # the better we can access our performance, 
            # this is especially important since many has little to none reference <- good compensation
            if i == 2:
                break
    except:
        citedByHrefDict[each] = "Nan"

print("Done with citedByHrefDict population")
timerPercentage = 1
for eachDictEntryKey in citedByHrefDict.keys():
    if eachDictEntryKey // 300 == 0:
        print(timerPercentage)
        timerPercentage+=1
    if citedByHrefDict[eachDictEntryKey] == 'Nan':
        for i in range(3):
            realisticCitation[eachDictEntryKey].append('Nan')
        continue
    for eachHref in citedByHrefDict[eachDictEntryKey]:
        tempRawPage = requests.get(customString + eachHref + refString).text
        soup = bs(tempRawPage, "html.parser")
        tempCitedByList = soup.find_all('li', class_= 'skip-numbering')
        for each in tempCitedByList: # tempCitedByList has all references on that article
            # there are cases where there are more than 1 anchor tag per entry
            tempHrefCheck = each.find_all('a', href = True)
            if(tempHrefCheck is not None):
                # this loop is here because the actual link might not be the first anchor tag
                if len(realisticCitation[eachDictEntryKey]) == 3:
                    break
                for eachTempHrefCheck in tempHrefCheck:
                    if eachTempHrefCheck["href"] == "/" + str(eachDictEntryKey) + "/":
                        realisticCitation[eachDictEntryKey].append(each.contents[0].strip().split("\n")[0])
            else:
                print(each, tempCitedByList)

print("Done with realisticCitation population")
for each in realisticCitation:
    if len(realisticCitation[each]) < 3:
        for i in range(3 - len(realisticCitation[each])):
            realisticCitation[each].append("Nan")

In [ ]:
realDf = pd.DataFrame(realisticCitation).T
# bigDf = pd.concat([OriDf, realDf],axis=1)
# bigDf = bigDf[~bigDf['PubMedIDs'].isin(errorList)]
# bigDf.reset_index(inplace=True)
# rawStringDf = pd.DataFrame(list(zip(ama,apa,mla,nlm)), columns = ["ama","apa","mla","nlm"])
# df = pd.concat([bigDf, rawStringDf],axis=1)
# df.drop(["index"], axis = 1, inplace=True)
# df.to_pickle("first30k.pkl")